<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# 模块 2.2: 组合逻辑
**上一章: [你的第一个 Chisel 模块](2.1_first_module.ipynb)**<br>
**下一章: [控制流](2.3_control_flow.ipynb)**

## 动机
在这一部分，您将看到如何使用 Chisel 组件来实现组合逻辑。
我们将演示如何连接和操作三种基本的 Chisel 类型：`UInt` - 无符号整数；`SInt` - 有符号整数；以及 `Bool` - 真或假。
请注意，所有 Chisel 变量都声明为 Scala `val`。
永远不要使用 Scala `var` 来表示硬件结构，因为一旦定义，结构本身就不会再改变；只有在运行硬件时其值可能会改变。
可以使用 Wires 来处理参数化类型。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# 常见运算符
现在您已经了解了如何构建`Module`，让我们制作一些硬件！看看下面的空模块。

class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

我们将类命名为 `MyModule`，并使其扩展 `Module`。这意味着它会映射到 Verilog 中的硬件模块。我们的 `MyModule` 模块有一个输入和一个输出。输入是一个 4 位无符号整数（`UInt`），输出也是如此。

<span style="color:blue">**示例：Scala 和 Chisel 运算符看起来相同**</span><br>
让我们看看可以对数据执行哪些不同操作。

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

我们创建了两个 `val`。第一个将两个 Scala `Int` 相加，因此 `println` 打印出整数 2。第二个 `val` 将两个 *Chisel* `UInt` 相加，因此 `println` 将其视为硬件节点并打印出类型名称和指针（`chisel3.core.UInt@d`）。请注意，`1.U` 是从 Scala `Int`（1）强制转换为 Chisel `UInt` 文字的类型转换。

我们需要将输出连接到某个东西，所以暂时将其连接到输入，就像上一个教程中的传递模块一样。

<span style="color:blue">**示例：不兼容的操作**</span><br>
如果我们将 Chisel `1.U` 添加到文字 `1` 中会发生什么？这些类型是不兼容的，因为前者是值为 1 的硬件线，而后者是值为 1 的 Scala 值。因此，Chisel 将给出类型不匹配错误。

In [ ]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModuleTwo))

在执行操作时记住类型之间的区别是很重要的。Scala 是一种强类型语言，因此任何类型转换都必须是显式的。

<span style="color:blue">**示例：更多 Chisel 运算符**</span><br>
其他常见的操作包括减法和乘法。这些操作在无符号整数上处理得如预期那样。让我们看看它们的实际操作。我们显示了 Verilog，尽管有一些基础的 Chisel 功能使我们期望的简单代码变得模糊。

In [ ]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(getVerilog(new MyOperators))

以下是上述操作的一个示例测试器。与上一个教程中使用匿名测试类不同，我们将创建一个显式测试类。这只是编写测试器的另一种选择方式。

In [ ]:
test(new MyOperators) {c =>
  c.io.out_add.expect(5.U)
  c.io.out_sub.expect(1.U)
  c.io.out_mul.expect(8.U)
}
println("SUCCESS!!")

<span style="color:blue">**示例：Mux 和连接**</span><br>
除了加法、减法和乘法之外，Chisel 还具有 mux 和连接运算符。下面显示了这些运算符。`Mux` 的操作类似于传统的三元运算符，具有顺序（选择，如果为真的值，如果为假的值）。请注意，`true.B` 和 `false.B` 是创建 Chisel Bool 文字的首选方法。`Cat` 的排序是 MSB 然后 LSB（其中 B 指的是位或位），只接受两个参数。连接超过两个值需要多次调用 `Cat` 或在后面的部分中涵盖的高级 Chisel 和 Scala 功能。

In [ ]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // 应返回3.U，因为s为true
  io.out_cat := Cat(2.U, 1.U)    // 连接2（b10）与1（b1），得到5（101）
}

println(getVerilog(new MyOperatorsTwo))

test(new MyOperatorsTwo) { c =>
  c.io.out_mux.expect(3.U)
  c.io.out_cat.expect(5.U)
}
println("SUCCESS!!")

注意Verilog中包含常量而不是实际的mux或连接逻辑。这是因为FIRRTL转换简化了电路，消除了明显的逻辑。

要获取更完整的Chisel运算符列表，请参阅[Chisel速查表](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)。要获取运算符及其实现细节的最完整列表，请查阅[Chisel API](https://chisel-lang.org/api/latest/)。

---
# 练习
要完成这些练习，您可能需要查看[Chisel速查表](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)。

<span style="color:red">**练习：MAC**</span><br>
创建一个Chisel模块，实现乘累加函数`(A*B)+C`，并通过测试台。

In [ ]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  ???
}

test(new MAC) { c =>
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    c.io.in_a.poke(in_a.U)
    c.io.in_b.poke(in_b.U)
    c.io.in_c.poke(in_c.U)
    c.io.out.expect((in_a * in_b + in_c).U)
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>解决方案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>

<span style="color:red">**练习：仲裁器**</span><br>
以下电路仲裁来自FIFO到两个并行处理单元。FIFO和处理元素（PEs）通过就绪-有效接口进行通信。构建仲裁器以将数据发送到任一PE，该PE准备接收数据，优先考虑PE0，如果两者都准备接收数据。请记住，仲裁器应在至少一个PE可以接收数据时告诉FIFO它准备好接收数据。同时，等待PE断言它准备好再断言数据有效。完成此练习可能需要使用二进制运算符。

<img src="images/arbiter.png" width="687" height="177">

In [ ]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })

  ???
}

test(new Arbiter) { c =>
  import scala.util.Random
  val data = Random.nextInt(65536)
  c.io.fifo_data.poke(data.U)
  
  for (i <- 0 until 8) {
    c.io.fifo_valid.poke((((i >> 0) % 2) != 0).B)
    c.io.pe0_ready.poke((((i >> 1) % 2) != 0).B)
    c.io.pe1_ready.poke((((i >> 2) % 2) != 0).B)

    c.io.fifo_ready.expect((i > 1).B)
    c.io.pe0_valid.expect((i == 3 || i == 7).B)
    c.io.pe1_valid.expect((i == 5).B)
    
    if (i == 3 || i ==7) {
      c.io.pe0_data.expect((data).U)
    } else if (i == 5) {
      c.io.pe1_data.expect((data).U)
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>解决方案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.fifo_ready := io.pe0_ready || io.pe1_ready
  io.pe0_valid := io.fifo_valid && io.pe0_ready
  io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_ready
  io.pe0_data := io.fifo_data
  io.pe1_data := io.fifo_data
</pre></article></div></section></div>

<span style="color:red">**练习：参数化加法器（可选）**</span><br>
这个可选的练习让您了解Chisel最强大的功能之一，即其参数化能力。为了演示这一点，我们将构建一个参数化的加法器，可以在溢出时饱和输出，或截断结果（即环绕）。

首先，看一下下面的`Module`。我们传递给它的参数称为`saturate`，其类型为*Scala* `Boolean`。这不是Chisel `Bool`。因此，我们不是创建一个可以饱和或截断的单个硬件加法器，而是创建一个*生成器*，它产生饱和硬件加法器*或*截断硬件加法器。决定在编译时进行。

接下来，注意输入和输出都是4位的`UInt`。Chisel具有内置的宽度推断，如果查看[速查表](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)，您将看到常规求和的位宽等于两个输入的最大位宽。这意味着

```scala
val sum = io.in_a + io.in_b
```

将使`sum`成为4位线，并且值将是4位输入的截断结果。要检查求和是否应该饱和，您需要将结果放入5位线中。这可以通过`+&`求和来完成，如速查表中所示。

```scala
val sum = io.in_a +& io.in_b
```

最后，请注意，将4位`UInt`线连接到5位`UInt`线将默认截断MSB。您可以使用这一点轻松截断非饱和加法器的5位和。

In [ ]:
class ParameterizedAdder(saturate: Boolean) extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

  ???
}

for (saturate <- Seq(true, false)) {
  test(new ParameterizedAdder(saturate)) { c =>
    // 100 random tests
    val cycles = 100
    import scala.util.Random
    import scala.math.min
    for (i <- 0 until cycles) {
      val in_a = Random.nextInt(16)
      val in_b = Random.nextInt(16)
      c.io.in_a.poke(in_a.U)
      c.io.in_b.poke(in_b.U)
      if (saturate) {
        c.io.out.expect(min(in_a + in_b, 15).U)
      } else {
        c.io.out.expect(((in_a + in_b) % 16).U)
      }
    }
    
    // ensure we test saturation vs. truncation
    c.io.in_a.poke(15.U)
    c.io.in_b.poke(15.U)
    if (saturate) {
      c.io.out.expect(15.U)
    } else {
      c.io.out.expect(14.U)
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>解决方案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
</pre></article></div></section></div>

---
# 完成！

[返回顶部。](#top)